In [1]:
import cma
import pickle
import numpy as np
import time

In [2]:
X = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['X']
y = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['y']
nb_data = np.size(y)
nb_data

69839

In [3]:
# so evol search is bounded by [0, 1]^12
assert np.max(X) <= 1.0
assert np.min(X) >= 0.0

In [4]:
# return id of COF closest to x_ask in feature space, which is not in the set of acquired COFs
def closest_COF(x_ask, ids_acquired):
    distances_to_xs = np.linalg.norm(X - x_ask, axis=1)
    assert np.size(distances_to_xs) == nb_data
    ids_sorted_by_closeness = np.argsort(distances_to_xs)
    for id_closest in ids_sorted_by_closeness:
        if not id_closest in ids_acquired:
            return id_closest

In [5]:
# two tests
assert closest_COF(X[3, :], []) == 3

ids = [i for i in range(nb_data) if not i == 3]
distances = np.linalg.norm(X - X[3, :], axis=1)
assert distances[closest_COF(X[3, :], [3])] == np.min(distances[ids])

48442

In [10]:
nb_iterations = 500

def evol_run(nb_iterations):
    x_init = X[np.random.choice(np.arange((nb_data)), size=1, replace=False)[0], :] # random initial COF

    # initialize evolutionary algo
    ids_acquired = []
    es = cma.CMAEvolutionStrategy(x0=x_init, sigma0=0.5, inopts={'bounds': [0.0, 1.0]})

    while len(ids_acquired) < nb_iterations:
        print("ask/tell sesh")
        # ask for a set of COFs to evaluate.
        xs_ask = es.ask()

        # find IDs of these COFs (the closest unacquired COFs in feature space to them.)
        ids_ask = []
        for x_ask in xs_ask:
            assert np.min(x_ask) >= 0.0
            assert np.max(x_ask) <= 1.0
            id_ask = closest_COF(x_ask, ids_acquired)
            ids_ask.append(id_ask)
            # officially acquire this COF
            ids_acquired.append(id_ask)

        # tell the ES algo about these most recently acquired COFs.
        es.tell(X[ids_ask, :], -y[ids_ask]) # cuz we wanna minimize

        print("\t# acquired COFs:", len(ids_acquired))
        print("\t# max y value: ", np.max(y[ids_acquired]))
    assert np.size(np.unique(ids_acquired[:nb_iterations]) == nb_iterations)
    return ids_acquired[:nb_iterations]

In [11]:
es_res = dict()
es_res['nb_runs']       = 10
es_res['nb_iterations'] = 250
es_res['ids_acquired'] = []
for r in range(es_res['nb_runs']):
    print("\n\nRUN", r)
    ids_acquired = evol_run(es_res['nb_iterations'])
    es_res['ids_acquired'].append(ids_acquired)
    
with open('es_results.pkl', 'wb') as file:
    pickle.dump(es_res, file)



RUN 0
(5_w,11)-aCMA-ES (mu_w=3.4,w_1=42%) in dimension 12 (seed=282932, Thu Jul  1 14:53:00 2021)
ask/tell sesh
	# acquired COFs: 11
	# max y value:  161.035530703
ask/tell sesh
	# acquired COFs: 22
	# max y value:  171.117194584
ask/tell sesh
	# acquired COFs: 33
	# max y value:  191.077676114
ask/tell sesh
	# acquired COFs: 44
	# max y value:  196.752963258
ask/tell sesh
	# acquired COFs: 55
	# max y value:  196.752963258
ask/tell sesh
	# acquired COFs: 66
	# max y value:  196.752963258
ask/tell sesh
	# acquired COFs: 77
	# max y value:  196.752963258
ask/tell sesh
	# acquired COFs: 88
	# max y value:  196.752963258
ask/tell sesh
	# acquired COFs: 99
	# max y value:  196.752963258
ask/tell sesh
	# acquired COFs: 110
	# max y value:  196.752963258
ask/tell sesh
	# acquired COFs: 121
	# max y value:  196.752963258
ask/tell sesh
	# acquired COFs: 132
	# max y value:  196.752963258
ask/tell sesh
	# acquired COFs: 143
	# max y value:  196.752963258
ask/tell sesh
	# acquired COFs: 154
	#